# (3) Manual Feature Scaling, Selection, and Encoding

**Feature scaling** and **feature_selection** are yet two components of a classical data science pipeline that isn't fully adressed in neither `featuretools` and `h2o`. `featuretools` has no mention of these, and `h2o` includes them in some models as hyperparameters to learn and doesn't include them in others. Whereas **data encoding** for categorical variables are handled intrinsically in `h2o` as we shall discuss in the next notebook,  it does so by applying different encodings for different algorithms. For example, XGBoost models perform an internal *one-hot encoding* and Gradient Boosting Machine (GBM) models perform *enum encoding*. For this reason, we will both demonstrate the case where we handle encoding ourselves and the case where we completely leave it to `h2o`.

In [68]:
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from utils import categorical_to_onehot_columns
from sklearn.preprocessing import LabelEncoder

SEED = 42
pd.options.mode.chained_assignment = None  # suppress SettingWithCopyWarning() for chained assignments

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style("white", {'ytick.major.size': 8.0})
sns.set_context("poster", font_scale=0.8)

## Reading Data

In [69]:
X_train = pd.read_csv('(2)data_automated_ops/train_users.csv')
X_train, Y_train = X_train.drop('country_destination', axis=1), X_train['country_destination']
X_train.head()

,gender,age,signup_method,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,...,buckets.CUM_SUM(CA),buckets.CUM_SUM(DE),buckets.CUM_SUM(FR),buckets.CUM_SUM(GB),buckets.CUM_SUM(AU),buckets.CUM_SUM(NL),buckets.CUM_SUM(US),buckets.CUM_SUM(IT),buckets.CUM_SUM(PT),buckets.CUM_SUM(ES)
0,FEMALE,31.000000,basic,en,direct,direct,omg,Web,Mac Desktop,Safari,...,12114.0,23369.0,21961.0,21458.0,8783.0,5457.0,119834.0,16818.0,3092.0,13463.0
1,FEMALE,43.857143,basic,en,direct,direct,untracked,Web,iPad,Mobile Safari,...,16926.0,33405.0,29925.0,29738.0,12100.0,7523.0,161807.0,25250.0,4714.0,21218.0
2,FEMALE,45.857143,basic,en,direct,direct,untracked,Web,Windows Desktop,Firefox,...,19322.0,39395.0,34357.0,34188.0,13713.0,8742.0,182625.0,30158.0,5525.0,25122.0
3,MALE,40.000000,basic,en,direct,direct,untracked,Web,Mac Desktop,Safari,...,18105.0,36038.0,32137.0,31839.0,12920.0,8105.0,171966.0,27678.0,5130.0,23270.0
4,FEMALE,33.714286,basic,en,direct,direct,untracked,Moweb,Windows Desktop,Chrome,...,12114.0,23369.0,21961.0,21458.0,8783.0,5457.0,119834.0,16818.0,3092.0,13463.0


In [70]:
X_test = pd.read_csv('(2)data_automated_ops/test_users.csv')
X_test.head()

,gender,age,signup_method,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,...,buckets.CUM_SUM(CA),buckets.CUM_SUM(DE),buckets.CUM_SUM(FR),buckets.CUM_SUM(GB),buckets.CUM_SUM(AU),buckets.CUM_SUM(NL),buckets.CUM_SUM(US),buckets.CUM_SUM(IT),buckets.CUM_SUM(PT),buckets.CUM_SUM(ES)
0,MALE,34.428571,basic,zh,seo,google,linked,Web,Mac Desktop,Chrome,...,13376.0,25996.0,23996.0,23648.0,9664.0,5960.0,130818.0,18632.0,3463.0,15211.0
1,FEMALE,23.000000,facebook,en,seo,facebook,linked,Web,iPhone,Mobile Safari,...,7178.0,13444.0,13753.0,12950.0,5352.0,3410.0,74974.0,10143.0,1837.0,8015.0
2,MALE,19.000000,basic,en,seo,google,untracked,Web,Mac Desktop,Safari,...,6019.0,11283.0,11806.0,10993.0,4565.0,2906.0,63880.0,8629.0,1562.0,6909.0
3,FEMALE,46.714286,basic,en,direct,direct,untracked,iOS,iPhone,Mobile Safari,...,19322.0,39395.0,34357.0,34188.0,13713.0,8742.0,182625.0,30158.0,5525.0,25122.0
4,MALE,41.000000,facebook,en,seo,facebook,linked,Web,Android Phone,Silk,...,18105.0,36038.0,32137.0,31839.0,12920.0,8105.0,171966.0,27678.0,5130.0,23270.0


## Feature Scaling
**Feature scaling** is notoriously effective in linear models and neural networks. As it is a rather conventional aspect of the data processing pipeline, we decided to include it here no matter what. An importing thing to note is that, we have to apply scaling (sometimes also referred to as **normalization**) only to *originally numerical* columns at first. Later in the notebook when we encode categorical variables, we will apply another scaling there as well. Let's first observe the different numeric and categorical variables we have.

In [71]:
numeric_variables = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_variables = [x for x in X_train.columns.tolist() if x not in numeric_variables]
print("NUMERIC VARIABLES: ", numeric_variables)
print("CATEGORICAL VARIABLES: ", categorical_variables)

NUMERIC VARIABLES:  ['age', 'signup_flow', 'LAST(sessions.secs_elapsed)', 'NUM_UNIQUE(sessions.action)', 'NUM_UNIQUE(sessions.action_type)', 'NUM_UNIQUE(sessions.action_detail)', 'NUM_UNIQUE(sessions.device_type)', 'SKEW(sessions.secs_elapsed)', 'MIN(sessions.secs_elapsed)', 'MEAN(sessions.secs_elapsed)', 'STD(sessions.secs_elapsed)', 'MAX(sessions.secs_elapsed)', 'MEDIAN(sessions.secs_elapsed)', 'HOUR(date_account_created)', 'HOUR(timestamp_first_active)', 'DAY(date_account_created)', 'DAY(timestamp_first_active)', 'WEEK(date_account_created)', 'WEEK(timestamp_first_active)', 'MONTH(date_account_created)', 'MONTH(timestamp_first_active)', 'YEAR(date_account_created)', 'YEAR(timestamp_first_active)', 'CUM_SUM(age)', 'buckets.CA', 'buckets.DE', 'buckets.FR', 'buckets.GB', 'buckets.AU', 'buckets.NL', 'buckets.US', 'buckets.IT', 'buckets.PT', 'buckets.ES', 'LAST(sessions.CUM_SUM(secs_elapsed))', 'SKEW(sessions.CUM_SUM(secs_elapsed))', 'MIN(sessions.CUM_SUM(secs_elapsed))', 'MEAN(sessions.

### Scaler Choice
You can check the most common feature scaling methods from [here](https://en.wikipedia.org/wiki/Feature_scaling). The most effective algorithms are **Min-Max Scaling**, **Mean Normalization**, **Gaussian (Standard) Scaling**, **Unit-Lenth Scaling**, **Robust Scaling**, **Logarithmic Scaling**, and **Exponential Scaling** depending on the application. Here, we went with a fairly safe method: Gaussian (Standard) scaling based on mean and standard deviation of the variable sample. 

In [72]:
scaler = StandardScaler()
scaler = scaler.fit(X_train[numeric_variables])
X_train[numeric_variables] = scaler.transform(X_train[numeric_variables])
X_test[numeric_variables] = scaler.transform(X_test[numeric_variables])
# Observe scaling effect
X_train.head()

C:\Users\UZAY\ANACONDA3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\UZAY\ANACONDA3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\UZAY\ANACONDA3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


,gender,age,signup_method,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,...,buckets.CUM_SUM(CA),buckets.CUM_SUM(DE),buckets.CUM_SUM(FR),buckets.CUM_SUM(GB),buckets.CUM_SUM(AU),buckets.CUM_SUM(NL),buckets.CUM_SUM(US),buckets.CUM_SUM(IT),buckets.CUM_SUM(PT),buckets.CUM_SUM(ES)
0,FEMALE,-0.644729,basic,en,direct,direct,omg,Web,Mac Desktop,Safari,...,-0.632065,-0.622391,-0.619435,-0.629440,-0.637644,-0.620349,-0.623034,-0.664049,-0.687217,-0.710146
1,FEMALE,0.504372,basic,en,direct,direct,untracked,Web,iPad,Mobile Safari,...,0.296425,0.237997,0.273689,0.285184,0.326004,0.250720,0.295706,0.283107,0.334959,0.342713
2,FEMALE,0.683121,basic,en,direct,direct,untracked,Web,Windows Desktop,Firefox,...,0.758740,0.751521,0.770716,0.776740,0.794609,0.764677,0.751388,0.834417,0.846047,0.872740
3,MALE,0.159642,basic,en,direct,direct,untracked,Web,Mac Desktop,Safari,...,0.523916,0.463725,0.521754,0.517265,0.564228,0.496104,0.518075,0.555842,0.597120,0.621303
4,FEMALE,-0.402141,basic,en,direct,direct,untracked,Moweb,Windows Desktop,Chrome,...,-0.632065,-0.622391,-0.619435,-0.629440,-0.637644,-0.620349,-0.623034,-0.664049,-0.687217,-0.710146


## Feature Selection
Not only that feature selection yields smaller training & test sets and decreases training time vastly, it also prevents models from overfitting and allows them to generalize better. Moreover, as different models react to high numbers of feature spaces differently, we decided that feature selection might allow us to compare these models in a more fair way.

### Setting Back Unknown Categorical Levels & Setting 0's for Numeric NaNs
In the previous notebook, we have converted the newly introduced missing values (from automated feature engineering) in all columns to NaNs for uniformity in representation. However, NaNs for numeric variables tell us that the corresponding users didn't have any related session information. For the majority of the numeric features we have generated, plugging in 0s as missing values seems to be logical here. On the other hand, NaNs for categorical variables tell us that corresponding users have untracked information or they used a tool/utility/method that is not recognized. Defining in a new categorical level of 'UNKNOWN' seems to be logical here. We also need these operations to eliminate missing values before applying feature selection algorithms.

In [73]:
X_train[categorical_variables] = X_train[categorical_variables].fillna('UNKNOWN')
X_test[categorical_variables] = X_test[categorical_variables].fillna('UNKNOWN')
X_train[numeric_variables] = X_train[numeric_variables].fillna(0.0)
X_test[numeric_variables] = X_test[numeric_variables].fillna(0.0)

### One-Hot Encoding Categorical Variables
Before we proceed any further, we will have to *temporarily* one-hot encode our categorical variables so that they fit in with our feature selection methods. (NOTE: Particulary, we are referring to the estimation of *chi-squared test statistic* and the utilization of *logistic regression* models.)

In [74]:
# Transform training set to one-hot encoded representation & get encoded columns
X_train_onehot_encoded = categorical_to_onehot_columns(df=X_train)
encoded_columns = X_train_onehot_encoded.columns.values.tolist()
# Transform test set to one-hot encoded representation
X_test_onehot_encoded = categorical_to_onehot_columns(df=X_test)
# Add categorical levels that exist in training set to test set
for fitted_column in encoded_columns:
    if fitted_column not in X_test_onehot_encoded.columns.values.tolist():
        X_test_onehot_encoded[fitted_column] = 0
# Drop categorical levels that don't exist in training set from test set
for column in X_test_onehot_encoded.columns.values.tolist():
    if column not in encoded_columns:
        X_test_onehot_encoded.drop(column, axis=1, inplace=True)
# Ensure that training and test sets have the same column-wise order
X_test_onehot_encoded = X_test_onehot_encoded[encoded_columns]
assert len(X_train_onehot_encoded.columns) == len(X_test_onehot_encoded.columns)

### Feature Selection Algorithms
Common feature selection algorithms include:
* **Pearson Correlation Coefficient**: Perphaps the most common and easiest way to perform feature selection. However, we will have to try different methods as our response variables are categorical rather than numerical. 
* **Chi-Squared Test Statistic**: The chi-square test is a statistical test of independence to determine the dependency of two variables. If the target variable is independent of the feature variable, we can discard that feature variable. If they are dependent, the feature variable is very important. It's a pretty conventional method.
* **Recursive Feature Elimination (RFE)**: Feature ranking with recursive feature elimination.
* **Variable Importances from a Baseline Model**: Applies common algorithms such as Random Forests, Logistic Regression, and XGBoost.

For the problem at hand, we have focused on the last three algorithms.

#### Chi-Squared Test Statistic

In [8]:
chi_model = SelectKBest(chi2, k=200)
chi_model.fit(X_train_onehot_encoded, Y_train)
chi_support = chi_model.get_support()
chi_selected_columns = X_train_onehot_encoded.loc[:, chi_support].columns.tolist()

#### Recursive Feature Elimination (RFE) by Logistic Regression (L2)

In [9]:
rfe_model = RFE(estimator=LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=150, 
                                             penalty='l2', random_state=SEED, n_jobs=-1), 
                n_features_to_select=200, step=100, verbose=5)
rfe_model.fit(X_train_onehot_encoded, Y_train)
rfe_support = rfe_model.get_support()
rfe_selected_columns = X_train_onehot_encoded.loc[:, rfe_support].columns.tolist()

Fitting estimator with 1031 features.
Fitting estimator with 931 features.
Fitting estimator with 831 features.
Fitting estimator with 731 features.
Fitting estimator with 631 features.
Fitting estimator with 531 features.
Fitting estimator with 431 features.
Fitting estimator with 331 features.
Fitting estimator with 231 features.


#### Variable Importances by Logistic Regression (L2)

In [12]:
lr_model = SelectFromModel(LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=150,
                                              penalty='l2', random_state=SEED, n_jobs=-1), 
                           threshold='1.25*median')
lr_model.fit(X_train_onehot_encoded, Y_train)
lr_support = lr_model.get_support()
lr_selected_columns = X_train_onehot_encoded.loc[:, lr_support].columns.tolist()

### Combining All Methods
By observing the *union* and *intersection* of the different features we have selected with the given methods above, we have decided that limiting the training & test sets to the *intersection* features seems like best option. This was partly because the *union* features were still large in number. It should be noted that the feature selection process adopted here is not optimized in any way. We could have used different *models*, *parameters*, and *numbers of independent features to select* in each method.

In [21]:
# Get the union of all features
all_selected_columns = list(set(chi_selected_columns + rfe_selected_columns + lr_selected_columns))
print("TOTAL NUMBER OF FEATURES SELECTED: ", len(all_selected_columns))
print("ALL SELECTED FEATURES: ", all_selected_columns)

TOTAL NUMBER OF FEATURES SELECTED:  483
ALL SELECTED FEATURES:  ['age_gender_bucket=55-59female', 'age_gender_bucket=45-49female', 'MODE(sessions.action)=personalize', 'buckets.LAST(sessions.action)=similar_listings', 'LAST(sessions.action)=new', 'buckets.LAST(sessions.action_detail)=view_search_results', 'buckets.MAX(sessions.secs_elapsed)', 'buckets.LAST(sessions.action_detail)=contact_host', 'buckets.LAST(sessions.action_detail)=change_trip_characteristics', 'buckets.LAST(users.first_affiliate_tracked)=omg', 'LAST(sessions.action)=set_user', 'LAST(sessions.action_type)=submit', 'CUM_SUM(NUM_UNIQUE(sessions.action))', 'gender=OTHER', 'first_device_type=Mac Desktop', 'LAST(sessions.action_type)=data', 'buckets.NUM_UNIQUE(users.signup_method)', 'buckets.MEDIAN(sessions.secs_elapsed)', 'age_gender_bucket=50-54female', 'affiliate_channel=content', 'buckets.LAST(users.first_browser)=IE', 'MODE(sessions.device_type)=Tablet', 'affiliate_channel=other', 'LAST(sessions.action)=search', 'LAST(

In [22]:
# Get the intersection of all features
common_selected_columns = list(set(chi_selected_columns).intersection(set(rfe_selected_columns), set(lr_selected_columns)))
print("NUMBER OF COMMON FEATURES SELECTED: ", len(common_selected_columns))
print("COMMON SELECTED FEATURES: ", common_selected_columns)

NUMBER OF COMMON FEATURES SELECTED:  99
COMMON SELECTED FEATURES:  ['buckets.MODE(users.first_browser)=Safari', 'age_gender_bucket=55-59female', 'MODE(sessions.action)=qt2', 'buckets.LAST(sessions.action)=index', 'signup_app=Android', 'affiliate_channel=api', 'LAST(sessions.device_type)=Android App Unknown Phone/Tablet', 'affiliate_provider=craigslist', 'first_device_type=Mac Desktop', 'LAST(sessions.action_type)=data', 'affiliate_provider=direct', 'affiliate_channel=content', 'buckets.LAST(users.first_browser)=IE', 'MAX(sessions.CUM_SUM(secs_elapsed))', 'MODE(sessions.action)=ajax_photo_widget_form_iframe', 'age_gender_bucket=20-24male', 'MIN(sessions.CUM_SUM(secs_elapsed))', 'buckets.LAST(users.first_browser)=Mobile Safari', 'MEAN(sessions.CUM_SUM(secs_elapsed))', 'affiliate_channel=other', 'MODE(sessions.action)=update', 'signup_app=Web', 'LAST(sessions.action)=UNKNOWN', 'first_affiliate_tracked=omg', 'buckets.MODE(users.first_browser)=Chrome', 'MAX(sessions.secs_elapsed)', 'MODE(se

The columns mentioned above are selected from the one-hot encoded representation of our training & test sets. We can convert them to base features corresponding to the original columns in our data when we read it in the beginning of this notebook. 

In [42]:
all_base_selected_columns = list(set([column.split('=')[0] for column in all_selected_columns]))
common_base_selected_columns = list(set([column.split('=')[0] for column in common_selected_columns]))
print("NUMBER OF BASE COLUMNS SELECTED: ", len(common_base_selected_columns))
print("BASE COLUMNS SELECTED: ", common_base_selected_columns)

NUMBER OF BASE COLUMNS SELECTED:  39
BASE COLUMNS SELECTED:  ['signup_flow', 'LAST(sessions.device_type)', 'MODE(sessions.device_type)', 'MAX(sessions.secs_elapsed)', 'CUM_SUM(STD(sessions.secs_elapsed))', 'affiliate_channel', 'buckets.LAST(sessions.action_detail)', 'NUM_UNIQUE(sessions.action_detail)', 'first_affiliate_tracked', 'age_gender_bucket', 'buckets.LAST(sessions.action)', 'buckets.LAST(users.affiliate_provider)', 'LAST(sessions.action_type)', 'first_browser', 'MODE(sessions.action_detail)', 'LAST(sessions.action)', 'MODE(sessions.action)', 'buckets.MODE(users.first_browser)', 'NUM_UNIQUE(sessions.action)', 'signup_app', 'CUM_SUM(LAST(sessions.secs_elapsed))', 'buckets.LAST(sessions.action_type)', 'signup_method', 'MEDIAN(sessions.CUM_SUM(secs_elapsed))', 'language', 'YEAR(date_account_created)', 'buckets.LAST(users.affiliate_channel)', 'MAX(sessions.CUM_SUM(secs_elapsed))', 'buckets.LAST(users.first_browser)', 'CUM_SUM(MIN(sessions.secs_elapsed))', 'MIN(sessions.CUM_SUM(secs

Now, let's create the trimmed versions of our data frames.

In [93]:
X_train_trimmed = X_train[common_base_selected_columns]
X_test_trimmed = X_test[common_base_selected_columns]
assert X_train_trimmed.shape[1] == X_test_trimmed.shape[1]
X_train_trimmed.head()

,signup_flow,LAST(sessions.device_type),MODE(sessions.device_type),MAX(sessions.secs_elapsed),CUM_SUM(STD(sessions.secs_elapsed)),affiliate_channel,buckets.LAST(sessions.action_detail),NUM_UNIQUE(sessions.action_detail),first_affiliate_tracked,age_gender_bucket,...,CUM_SUM(MIN(sessions.secs_elapsed)),MIN(sessions.CUM_SUM(secs_elapsed)),MODE(sessions.action_type),LAST(sessions.CUM_SUM(secs_elapsed)),affiliate_provider,MEAN(sessions.CUM_SUM(secs_elapsed)),first_device_type,LAST(sessions.action_detail),buckets.LAST(sessions.device_type),NUM_UNIQUE(sessions.action_type)
0,-0.428195,Mac Desktop,Mac Desktop,0.228909,-1.729338,direct,p3,0.361036,omg,30-34female,...,-1.616536,0.593651,view,0.577406,direct,0.593643,Mac Desktop,wishlist_content_update,Mac Desktop,0.748889
1,-0.428195,UNKNOWN,UNKNOWN,0.000000,0.000000,direct,user_profile,0.000000,untracked,40-44female,...,0.000000,0.000000,UNKNOWN,0.000000,direct,0.000000,iPad,UNKNOWN,UNKNOWN,0.000000
2,-0.428195,UNKNOWN,UNKNOWN,0.000000,0.000000,direct,message_thread,0.000000,untracked,45-49female,...,0.000000,0.000000,UNKNOWN,0.000000,direct,0.000000,Windows Desktop,UNKNOWN,iPad Tablet,0.000000
3,-0.428195,UNKNOWN,UNKNOWN,0.000000,0.000000,direct,UNKNOWN,0.000000,untracked,40-44male,...,0.000000,0.000000,UNKNOWN,0.000000,direct,0.000000,Mac Desktop,UNKNOWN,Mac Desktop,0.000000
4,2.710926,UNKNOWN,UNKNOWN,0.000000,0.000000,direct,p3,0.000000,untracked,30-34female,...,0.000000,0.000000,UNKNOWN,0.000000,direct,0.000000,Windows Desktop,UNKNOWN,Mac Desktop,0.000000


## Save Progress with Raw & Trimmed Data

In [94]:
# Add back the response variable to sets & save progress
X_train_trimmed.loc[:, 'country_destination'] = Y_train.loc[:]
X_train_trimmed.to_csv('(3)data_trimmed/raw/train_users.csv', index=None)
X_test_trimmed.to_csv('(3)data_trimmed/raw/test_users.csv', index=None)
# Drop response variable again, as we will continue processing features
X_train_trimmed.drop('country_destination', axis=1, inplace=True)

C:\Users\UZAY\ANACONDA3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\UZAY\ANACONDA3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## Encoding Categorical Variables
Although we had chosen **one-hot encoding** as our strategy previously, those encodings were solely used for the purposes of feature selection and abondoned afterwards. Besides, there exists other and more efficient strategies in the literature that can be explored:

* **Labeled Encoding**: Interprets categories as ordered integers. The extracted ordinality is almost always wrong, hence this is not really a preferred method.
* **Frequency Encoding**: Encodes categorical levels of each feature to values between 0.0 and 1.0 based on their relative frequincy. This method especially works when there is a high number of categorical levels that are somewhat imbalanced in distribution.
* **Target Mean Encoding**: Encodes categorical levels of each feature to the mean of the response. This method works best with binary classification, but it often yields *data leakage*.

It should be noted that one-hot encoding variables increases the feature space vastly, and this may decrease the potential performance of the *tree-based models*. This is why we don't want to use this type of encoding to get our final data form. Instead, we have chosen to go with labeled encoding as an experiment. As previously discussed, `h2o`'s automated modelling method applies different kind of encodings, but labeled encoding was one strategy that wasn't automatically applied.

In [95]:
remaining_categorical_vars = []
for column in X_train_trimmed.columns.values.tolist():
    if not is_numeric_dtype(X_train_trimmed[column]):
        print("Currently encoding column: ", column)
        remaining_categorical_vars.append(column)
        
        encoder = LabelEncoder()
        encoder.fit(X_train_trimmed[column])
        available_levels = list(encoder.classes_)
        for test_level in set(X_test_trimmed[column].values.tolist()):
            if test_level not in available_levels:
                X_test_trimmed.loc[X_test_trimmed[column] == test_level, column] = X_train_trimmed[column].mode()[0]
        
        X_train_trimmed.loc[:, column] = encoder.transform(X_train_trimmed[column])
        X_test_trimmed.loc[:, column] = encoder.transform(X_test_trimmed[column])

Currently encoding column:  LAST(sessions.device_type)
Currently encoding column:  MODE(sessions.device_type)
Currently encoding column:  affiliate_channel
Currently encoding column:  buckets.LAST(sessions.action_detail)
Currently encoding column:  first_affiliate_tracked
Currently encoding column:  age_gender_bucket
Currently encoding column:  buckets.LAST(sessions.action)
Currently encoding column:  buckets.LAST(users.affiliate_provider)
Currently encoding column:  LAST(sessions.action_type)
Currently encoding column:  first_browser
Currently encoding column:  MODE(sessions.action_detail)
Currently encoding column:  LAST(sessions.action)
Currently encoding column:  MODE(sessions.action)
Currently encoding column:  buckets.MODE(users.first_browser)
Currently encoding column:  signup_app
Currently encoding column:  buckets.LAST(sessions.action_type)
Currently encoding column:  signup_method
Currently encoding column:  language
Currently encoding column:  buckets.LAST(users.affiliate_ch

In [96]:
X_train_trimmed.head()

,signup_flow,LAST(sessions.device_type),MODE(sessions.device_type),MAX(sessions.secs_elapsed),CUM_SUM(STD(sessions.secs_elapsed)),affiliate_channel,buckets.LAST(sessions.action_detail),NUM_UNIQUE(sessions.action_detail),first_affiliate_tracked,age_gender_bucket,...,CUM_SUM(MIN(sessions.secs_elapsed)),MIN(sessions.CUM_SUM(secs_elapsed)),MODE(sessions.action_type),LAST(sessions.CUM_SUM(secs_elapsed)),affiliate_provider,MEAN(sessions.CUM_SUM(secs_elapsed)),first_device_type,LAST(sessions.action_detail),buckets.LAST(sessions.device_type),NUM_UNIQUE(sessions.action_type)
0,-0.428195,5,5,0.228909,-1.729338,2,7,0.361036,3,11,...,-1.616536,0.593651,7,0.577406,4,0.593643,3,110,1,0.748889
1,-0.428195,7,7,0.000000,0.000000,2,14,0.000000,6,17,...,0.000000,0.000000,0,0.000000,4,0.000000,7,0,2,0.000000
2,-0.428195,7,7,0.000000,0.000000,2,6,0.000000,6,20,...,0.000000,0.000000,0,0.000000,4,0.000000,6,0,4,0.000000
3,-0.428195,7,7,0.000000,0.000000,2,0,0.000000,6,18,...,0.000000,0.000000,0,0.000000,4,0.000000,3,0,1,0.000000
4,2.710926,7,7,0.000000,0.000000,2,7,0.000000,6,11,...,0.000000,0.000000,0,0.000000,4,0.000000,6,0,1,0.000000


## Feature Scaling for Remaining Categorical Variables

After this step, both our training & test sets are fully scaled and ready for being passed on to predictive models.

In [97]:
scaler = StandardScaler()
scaler = scaler.fit(X_train_trimmed[remaining_categorical_vars])
X_train_trimmed.loc[:, remaining_categorical_vars] = scaler.transform(X_train_trimmed.loc[:, remaining_categorical_vars])
X_test_trimmed.loc[:, remaining_categorical_vars] = scaler.transform(X_test_trimmed.loc[:, remaining_categorical_vars])

X_train_trimmed.head()

C:\Users\UZAY\ANACONDA3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\UZAY\ANACONDA3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int32 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\UZAY\ANACONDA3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\UZAY\ANACONDA3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int32 were all converted to float64 by StandardScaler.
  after remo

,signup_flow,LAST(sessions.device_type),MODE(sessions.device_type),MAX(sessions.secs_elapsed),CUM_SUM(STD(sessions.secs_elapsed)),affiliate_channel,buckets.LAST(sessions.action_detail),NUM_UNIQUE(sessions.action_detail),first_affiliate_tracked,age_gender_bucket,...,CUM_SUM(MIN(sessions.secs_elapsed)),MIN(sessions.CUM_SUM(secs_elapsed)),MODE(sessions.action_type),LAST(sessions.CUM_SUM(secs_elapsed)),affiliate_provider,MEAN(sessions.CUM_SUM(secs_elapsed)),first_device_type,LAST(sessions.action_detail),buckets.LAST(sessions.device_type),NUM_UNIQUE(sessions.action_type)
0,-0.428195,-1.245941,-1.251286,0.228909,-1.729338,-0.508442,-0.569216,0.361036,-0.397753,-0.611161,...,-1.616536,0.593651,2.199263,0.577406,-0.589559,0.593643,-0.928579,2.580894,-0.809545,0.748889
1,-0.428195,0.023564,0.025911,0.000000,0.000000,-0.508442,0.773677,0.000000,0.832197,0.406107,...,0.000000,0.000000,-0.575552,0.000000,-0.589559,0.000000,1.168565,-0.529941,-0.129710,0.000000
2,-0.428195,0.023564,0.025911,0.000000,0.000000,-0.508442,-0.761058,0.000000,0.832197,0.914741,...,0.000000,0.000000,-0.575552,0.000000,-0.589559,0.000000,0.644279,-0.529941,1.229960,0.000000
3,-0.428195,0.023564,0.025911,0.000000,0.000000,-0.508442,-1.912109,0.000000,0.832197,0.575652,...,0.000000,0.000000,-0.575552,0.000000,-0.589559,0.000000,-0.928579,-0.529941,-0.809545,0.000000
4,2.710926,0.023564,0.025911,0.000000,0.000000,-0.508442,-0.569216,0.000000,0.832197,-0.611161,...,0.000000,0.000000,-0.575552,0.000000,-0.589559,0.000000,0.644279,-0.529941,-0.809545,0.000000


## Save Progress with Label Encoded & Trimmed Data

In [99]:
# Add back the response variable to sets & save progress
X_train_trimmed.loc[:, 'country_destination'] = Y_train.loc[:]
X_train_trimmed.to_csv('(3)data_trimmed/label_encoded/train_users.csv', index=None)
X_test_trimmed.to_csv('(3)data_trimmed/label_encoded/test_users.csv', index=None)